####  https://www.tensorflow.org/tutorials/sequences/text_generation

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
print(tf.__version__)

tf.enable_eager_execution()

import numpy as np
import os
import time

import numpy as np
import random
import sys
import io
import pandas as pd



1.13.0-dev20181220


### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
root_path = "../"
path_data_clean = root_path + "data/clean/"
dfNoticias = pd.read_pickle(path_data_clean + "/dfNoticiasCleanV2.p")
#dfNoticias.info()

In [3]:
raw_text = ""
count = 0
for index, row in dfNoticias.iterrows():    
    if index < 800: #max aporx 3500
        raw_text += row["Cuerpo"]
        
text = raw_text.lower()

### Read the data

First, look in the text.

In [4]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1277689 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])


la presidenta argentina, cristina fernández, tomó juramento a los tres nuevos ministros de su gabinete que designó el pasado lunes, en su primer acto público tras regresar a las funciones de gobierno tras 40 días de reposo por una neurocirugía.
fern


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

90 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\nla president' ---- characters mapped to int ---- > [ 0 45 34  1 49 51 38 52 42 37 38 47 53]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. 

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.


l
a
 
p


The `batch` method lets us easily convert these individual characters to sequences of the desired size. 

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\nla presidenta argentina, cristina fernández, tomó juramento a los tres nuevos ministros de su gabine'
'te que designó el pasado lunes, en su primer acto público tras regresar a las funciones de gobierno t'
'ras 40 días de reposo por una neurocirugía.\nfernández, que alivió el riguroso luto que vestía desde l'
'a muerte de su esposo, el ex presidente néstor kirchner, en 2010, había aparecido en un acto en la ca'
'sa rosada por última vez el 4 de octubre, en vísperas de la intervención a la que se sometió por un h'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\nla presidenta argentina, cristina fernández, tomó juramento a los tres nuevos ministros de su gabin'
Target data: 'la presidenta argentina, cristina fernández, tomó juramento a los tres nuevos ministros de su gabine'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 45 ('l')
Step    1
  input: 45 ('l')
  expected output: 34 ('a')
Step    2
  input: 34 ('a')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 49 ('p')
Step    4
  input: 49 ('p')
  expected output: 51 ('r')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [15]:
# Batch size 
BATCH_SIZE = 64 #64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

Use `CuDNNGRU` if running on GPU.  

In [17]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [19]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:


## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [20]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 90) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length: 

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23040     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 90)            92250     
Total params: 4,050,522
Trainable params: 4,050,522
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. 

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [22]:
print(tf.__version__)


1.13.0-dev20181220


In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [24]:
sampled_indices

array([77, 27, 31, 44, 78, 11, 36, 43,  2, 41, 51,  8, 73, 31, 55, 72, 17,
       16, 64, 72, 63, 21,  7, 89, 48, 41, 26, 78,  1, 73, 85, 29,  9, 51,
       85, 26, 89, 89, 37,  3, 38, 54,  0, 49, 31, 28, 15, 45, 20, 32, 58,
       68, 44, 75, 82, 72,  2,  1, 23, 75, 33, 68, 70, 61, 61, 20, 70,  7,
        1, 28,  3, 56, 89, 31, 22, 32, 42,  2, 15,  7, 14, 37, 69,  9,  7,
       48, 41, 85, 50,  8,  0, 55, 66, 88,  8, 35, 62, 83, 20, 63])

Decode these to see the text predicted by this untrained model:

In [25]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'iones y las conversaciones mantenidas con todas las partes creemos que esta es la mejor solución par'

Next Char Predictions: 
 'ó:[kö*cj!hr\'í[vé0/°é\xad4&…oh9ö í—?(r—9……d"eu\np[;.l3]yákñÿé! 6ñ_áæ¡¡3æ& ;"w…[5]i!.&-dã(&oh—q\'\nvº”\'bªć3\xad'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions. 

Because our model returns logits, we need to set the `from_logits` flag. 


In [26]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 90)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.498343


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function. 

In [27]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

filepath="weights/weights.besttf.hdf5"
checkpointBest = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [29]:
# def on_epoch_end(epoch, _):
#     # Function invoked at end of each epoch. Prints generated text.
#     print()
#     print('----- Generating text after Epoch: %d' % epoch)

#     start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0]:
#     #for diversity in [0.5]:
#         print('\n\n\n')
#         print('----- diversidad:', diversity)
#         print('\n\n')

#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         #generated += sentence
#         print('----- semilla: "' + sentence + '"')
#         print('\n\n')
#         sys.stdout.write(generated)

#         for i in range(300):
#             x_pred = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t, char_indices[char]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()

# # updatable plot
# # a minimal example (sort of)
# import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="train_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()    
        




### Execute the training

To keep this quick, train the model for just 3 epochs:

In [30]:
EPOCHS=60

model.load_weights("weights/weights.besttf.hdf5")


In [32]:

#, plot_losses
history = model.fit(dataset.repeat(), 
                    epochs=EPOCHS,
                    steps_per_epoch=steps_per_epoch, 
                    callbacks=[checkpoint_callback , checkpointBest])

Epoch 1/60
198/199 [============================>.] - ETA: 2s - loss: 1.2468
Epoch 00001: loss improved from 1.39728 to 1.24649, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 429s 2s/step - loss: 1.2465
Epoch 2/60
198/199 [============================>.] - ETA: 2s - loss: 1.1923
Epoch 00002: loss improved from 1.24649 to 1.19180, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 423s 2s/step - loss: 1.1918
Epoch 3/60
198/199 [============================>.] - ETA: 2s - loss: 1.1440
Epoch 00003: loss improved from 1.19180 to 1.14361, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 414s 2s/step - loss: 1.1436
Epoch 4/60
198/199 [============================>.] - ETA: 2s - loss: 1.0998
Epoch 00004: loss improved from 1.14361 to 1.09959, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 406s 2s/step - loss: 1.0996
Epoch 5/60
198/199 [====

199/199 [==============================] - 406s 2s/step - loss: 0.7896
Epoch 16/60
198/199 [============================>.] - ETA: 2s - loss: 0.7747
Epoch 00016: loss improved from 0.78958 to 0.77438, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 406s 2s/step - loss: 0.7744
Epoch 17/60
198/199 [============================>.] - ETA: 2s - loss: 0.7607
Epoch 00017: loss improved from 0.77438 to 0.76049, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 407s 2s/step - loss: 0.7605
Epoch 18/60
198/199 [============================>.] - ETA: 2s - loss: 0.7483
Epoch 00018: loss improved from 0.76049 to 0.74815, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 405s 2s/step - loss: 0.7481
Epoch 19/60
198/199 [============================>.] - ETA: 2s - loss: 0.7362
Epoch 00019: loss improved from 0.74815 to 0.73600, saving model to weights/weights.besttf.hdf5
199/199 [===========

198/199 [============================>.] - ETA: 2s - loss: 0.6615
Epoch 00030: loss improved from 0.66408 to 0.66142, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 407s 2s/step - loss: 0.6614
Epoch 31/60
198/199 [============================>.] - ETA: 2s - loss: 0.6541
Epoch 00031: loss improved from 0.66142 to 0.65413, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 405s 2s/step - loss: 0.6541
Epoch 32/60
198/199 [============================>.] - ETA: 2s - loss: 0.6523
Epoch 00032: loss improved from 0.65413 to 0.65222, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 405s 2s/step - loss: 0.6522
Epoch 33/60
198/199 [============================>.] - ETA: 2s - loss: 0.6495
Epoch 00033: loss improved from 0.65222 to 0.64940, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 407s 2s/step - loss: 0.6494
Epoch 34/60
198/199 [===========

199/199 [==============================] - 412s 2s/step - loss: 0.6177
Epoch 46/60
198/199 [============================>.] - ETA: 2s - loss: 0.6179
Epoch 00046: loss did not improve from 0.61767
199/199 [==============================] - 411s 2s/step - loss: 0.6179
Epoch 47/60
198/199 [============================>.] - ETA: 2s - loss: 0.6178
Epoch 00047: loss improved from 0.61767 to 0.61765, saving model to weights/weights.besttf.hdf5
199/199 [==============================] - 410s 2s/step - loss: 0.6177
Epoch 48/60
198/199 [============================>.] - ETA: 2s - loss: 0.6184
Epoch 00048: loss did not improve from 0.61765
199/199 [==============================] - 410s 2s/step - loss: 0.6184
Epoch 49/60
198/199 [============================>.] - ETA: 2s - loss: 0.6179
Epoch 00049: loss did not improve from 0.61765
199/199 [==============================] - 410s 2s/step - loss: 0.6180
Epoch 50/60
198/199 [============================>.] - ETA: 2s - loss: 0.6165
Epoch 00050: loss 

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.



Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # You can change the start string to experiment
  #start_string = 'c'

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="colo colo"))

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output. 

We will use `tf.GradientTape` to track the gradiends. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.

